In [1]:
import pandas as pd
import numpy as np
import collections


In [2]:
def load(path):
	df = None
	df = pd.read_csv(path)
	return df

In [3]:
def prior(df):
    ham_prior = 0
    spam_prior =  0

    ham_prior = df['label'].value_counts()[1]/df.shape[0]
    spam_prior = df['label'].value_counts()[0]/df.shape[0]  
    
    print(df['label'].value_counts())
    
    return ham_prior, spam_prior

In [4]:
def likelihood(df):
    ham_like_dict = {}
    spam_like_dict = {}
    '''YOUR CODE HERE'''
    spam_counter = collections.Counter()
    ham_counter = collections.Counter()
    #create list of all spam text
    seriesOfAllSpam = df.loc[df['label'] == 'spam']
    listOfAllSpamText = seriesOfAllSpam['text'].tolist()
    
    #create list of all ham text
    seriesOfAllHam = df.loc[df['label'] == 'ham']
    listOfAllHamText = seriesOfAllHam['text'].tolist()
    
    for i in range(len(listOfAllSpamText)):
        wordList = listOfAllSpamText[i].split(" ")
        unique = set(wordList)
        counterTemp = collections.Counter(list(unique))
        spam_counter.update(counterTemp)
        
    for i in range(len(listOfAllSpamText)):
        wordList = listOfAllHamText[i].split(" ")
        unique = set(wordList)
        counterTemp = collections.Counter(list(unique))
        ham_counter.update(counterTemp)
    
    ham_like_dict = {k:v/1199 for (k,v) in ham_counter.items()}
    spam_like_dict = {k:v/1199 for (k,v) in spam_counter.items()}

    return ham_like_dict, spam_like_dict


In [5]:
def predict(ham_prior, spam_prior, ham_like_dict, spam_like_dict, text):
    '''
    prediction function that uses prior and likelihood structure to compute proportional posterior for a single line of text
    '''
    #ham_spam_decision = 1 if classified as spam, 0 if classified as normal/ham
    ham_spam_decision = None




    '''YOUR CODE HERE'''
    alpha = 2
    #values become too small over time
    #ham_posterior = posterior probability that the email is normal/ham
    ham_posterior = None
    ham_posterior = 1
    for i in text.split():
        if i in ham_like_dict:
            ham_posterior *= (ham_like_dict[i] + alpha)
    ham_posterior *= ham_prior
    
    

    #spam_posterior = posterior probability that the email is spam
    spam_posterior = None
    spam_posterior = 1
    for i in text.split():
        if i in spam_like_dict:
            spam_posterior *= (spam_like_dict[i] + alpha)
    spam_posterior *= spam_prior
    
    
    spam_post = ((spam_posterior)/(ham_posterior + spam_posterior))
    ham_post = ((ham_posterior)/(ham_posterior + spam_posterior))
    if ((spam_post > ham_post)):
        ham_spam_decision = 1
    else:
        ham_spam_decision = 0
    
    '''END'''
    return ham_spam_decision




In [6]:
def metrics(ham_prior, spam_prior, ham_dict, spam_dict, df):

    hh = 0 #true negatives, truth = ham, predicted = ham
    hs = 0 #false positives, truth = ham, pred = spam
    sh = 0 #false negatives, truth = spam, pred = ham
    ss = 0 #true positives, truth = spam, pred = spam
    num_rows = df.shape[0]
    for i in range(num_rows):
        roi = df.iloc[i,:]
        roi_text = roi.text
        roi_label = roi.label_num
        guess = predict(ham_prior, spam_prior, ham_dict, spam_dict, roi_text)
        if roi_label == 0 and guess == 0:
            hh += 1
        elif roi_label == 0 and guess == 1:
            hs += 1
        elif roi_label == 1 and guess == 0:
            sh += 1
        elif roi_label == 1 and guess == 1:
            ss += 1
    
    acc = (ss + hh)/(ss+hh+sh+hs)
    precision = (ss)/(ss + hs)
    recall = (ss)/(ss + sh)
    return acc, precision, recall
    
if __name__ == "__main__":
    ham_prior, spam_prior = prior(load("./TRAIN_balanced_ham_spam.csv"))
    ham_dict, spam_dict = likelihood(load("./TRAIN_balanced_ham_spam.csv"))
    
    df = load("./TEST_balanced_ham_spam.csv")
    print(metrics(ham_prior, spam_prior, ham_dict, spam_dict, df))
    

ham     1199
spam    1199
Name: label, dtype: int64
(0.9633333333333334, 0.9826388888888888, 0.9433333333333334)


C:\Users\Jonat\AppData\Local\Temp/ipykernel_12896/3223532537.py:33: RuntimeWarning: invalid value encountered in double_scalars
  spam_post = ((spam_posterior)/(ham_posterior + spam_posterior))
C:\Users\Jonat\AppData\Local\Temp/ipykernel_12896/3223532537.py:34: RuntimeWarning: invalid value encountered in double_scalars
  ham_post = ((ham_posterior)/(ham_posterior + spam_posterior))
